### MSDS 696 Notebook4 Create SQLite Database, load data, and perform Basic EDA via SQL queries

### Project Title:
Create and Build A Data Engineering Pipeline to Collect, Process, and Store Spotify Data. This is intended to be a fun project to look at who the most popular artists are, what their most popular tracks are, and look at some characteristics of the songs.

### Mary J Hollon

### Due 8-22-2024

### Note Book Purpose:

The purpose of this notebook is to complete any remaining data cleaning, before creating an SQLite database, loading the data and performing some queries for basic EDA.

Let's start by loading a data file and examining it

In [1]:
#import necessary files

import pandas as pd
import os

# Load the CSV files into DataFrames
artists_df = pd.read_csv('artists_2015_genre_new.csv')
tracks_df = pd.read_csv('processed_tracks_2015.csv')



In [2]:
# Display the first few rows of the artists DataFrame 

artists_df.head()


,id,name,popularity,genres,year,simplified_genre
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift,100,['pop'],2015,Pop
1,3TVXtAsR1Inumwj472S9r4,Drake,95,"['canadian hip hop', 'canadian pop', 'hip hop'...",2015,Hip Hop
2,4oUHIQIBe0LHzYfvXNW4QM,Morgan Wallen,91,['contemporary country'],2015,Country
3,2YZyLoL8N0Wb9xBt1NhZWg,Kendrick Lamar,92,"['conscious hip hop', 'hip hop', 'rap', 'west ...",2015,Hip Hop
4,5K4W6rqBFWDnAN6FQUkS6x,Kanye West,92,"['chicago rap', 'hip hop', 'rap']",2015,Rap


Let's rename some columns in the artists data files

In [3]:
def save_processed_artist_files(start_year, end_year):
    
    for year in range(start_year, end_year + 1):
        # Load the file for the current year
        file_name = f'artists_{year}_genre_new.csv'
        artists_df = pd.read_csv(file_name)
        
        # Rename the columns 'id' to 'artist_id' and 'name' to 'artist_name'
        artists_df_renamed = artists_df.rename(columns={'id': 'artist_id', 'name': 'artist_name'})
        
        # Save the file with the appropriate name
        processed_file_name = f'processed_artist_{year}.csv'
        artists_df_renamed.to_csv(processed_file_name, index=False)
        print(f"Saved: {processed_file_name}")

# Run the function for the years 2015 to 2024
save_processed_artist_files(2015, 2024)


Saved: processed_artist_2015.csv
Saved: processed_artist_2016.csv
Saved: processed_artist_2017.csv
Saved: processed_artist_2018.csv
Saved: processed_artist_2019.csv
Saved: processed_artist_2020.csv
Saved: processed_artist_2021.csv
Saved: processed_artist_2022.csv
Saved: processed_artist_2023.csv
Saved: processed_artist_2024.csv


In [4]:
# let's look at one

df = pd.read_csv('processed_artist_2018.csv')
df.head()

,artist_id,artist_name,popularity,genres,year,simplified_genre
0,6JMGrupbzJZ3yuQhTGyeHr,Year 200X,15,['scorecore'],2018,Other
1,06HL4z0CvFAxyc27GXpf02,Taylor Swift,100,['pop'],2018,Pop
2,3TVXtAsR1Inumwj472S9r4,Drake,95,"['canadian hip hop', 'canadian pop', 'hip hop'...",2018,Hip Hop
3,4oUHIQIBe0LHzYfvXNW4QM,Morgan Wallen,91,['contemporary country'],2018,Country
4,2YZyLoL8N0Wb9xBt1NhZWg,Kendrick Lamar,92,"['conscious hip hop', 'hip hop', 'rap', 'west ...",2018,Hip Hop


The column "id" has been renamed to "artist_id" and the column "name" has been renamed to "artist_name" in the artist data. I am doing this to avoid confusion in the database tables.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   artist_id         400 non-null    object
 1   artist_name       400 non-null    object
 2   popularity        400 non-null    int64 
 3   genres            400 non-null    object
 4   year              400 non-null    int64 
 5   simplified_genre  400 non-null    object
dtypes: int64(2), object(4)
memory usage: 18.9+ KB


Now, let's check and rename some columns in the track data files

In [6]:
tracks_df.head()

,id,name,artist_id,year,popularity,release_date,energy,danceability,instrumentalness,loudness,...,valence,energy_category,tempo_category,danceability_category,valence_category,danceability_valence_interaction,loudness_scaled,loudness_category,instrumentalness_category,custom_score
0,3fqwjXwUGN6vbzIwvyFMhx,Tennessee Whiskey,4YLtscXsxbVgi031ovDDdh,2015,83,2015-05-04,0.370,0.392,0.00960,-10.888,...,0.512,Medium,Slow,Low,Neutral,0.200704,0.688470,High,Low,0.475794
1,3pXF1nA74528Edde4of9CC,Don't,2EMAnMvWE2eb56ToJVfCWs,2015,83,2015-10-02,0.356,0.765,0.00000,-5.556,...,0.189,Medium,Slow,High,Sad,0.144585,0.870686,Very High,None,0.539887
2,0QZ5yyl6B6utIWkxeBDxQN,The Night We Met,6ltzsmQQbmdoHHbLZ4ZN25,2015,78,2015-04-07,0.366,0.545,0.26700,-9.510,...,0.100,Medium,Slow,Medium,Sad,0.054500,0.735561,High,Low,0.427112
3,6K4t31amVTZDgR3sKmwUJJ,The Less I Know The Better,5INjqkS1o8h1imAzPqGZBb,2015,85,2015-07-17,0.740,0.640,0.00678,-4.083,...,0.785,High,Medium,Medium,Happy,0.502400,0.921024,Very High,Low,0.757455
4,43PuMrRfbyyuz4QpZ3oAwN,Exchange,2EMAnMvWE2eb56ToJVfCWs,2015,81,2015-10-02,0.433,0.525,0.00000,-10.598,...,0.276,Medium,Fast,Medium,Sad,0.144900,0.698380,High,None,0.474426


We will rename come columns and drop some columns.
 - rename "id" to "track_id" and "name" to "track_name"
 - drop "popularity' and "release date" columns because I will not use them  

In [7]:
def save_updated_tracks_files(start_year, end_year):
    
    for year in range(start_year, end_year + 1):
        
        # Load the file for the current year
        file_name = f'processed_tracks_{year}.csv'
        tracks_df = pd.read_csv(file_name)
        
        # Rename the columns 'id' to 'track_id', 'name' to 'track_name', and drop the 'popularity' and release_date columns
        tracks_df_renamed = tracks_df.rename(columns={'id': 'track_id', 'name': 'track_name'})
        tracks_df_renamed = tracks_df_renamed.drop(columns=['popularity','release_date'])
        
        # Save the file with the appropriate name
        processed_file_name = f'updated_processed_tracks_{year}.csv'
        tracks_df_renamed.to_csv(processed_file_name, index=False)
        print(f"Saved: {processed_file_name}")

# Run the function for the years 2015 to 2024
save_updated_tracks_files(2015, 2024)


Saved: updated_processed_tracks_2015.csv
Saved: updated_processed_tracks_2016.csv
Saved: updated_processed_tracks_2017.csv
Saved: updated_processed_tracks_2018.csv
Saved: updated_processed_tracks_2019.csv
Saved: updated_processed_tracks_2020.csv
Saved: updated_processed_tracks_2021.csv
Saved: updated_processed_tracks_2022.csv
Saved: updated_processed_tracks_2023.csv
Saved: updated_processed_tracks_2024.csv


In [8]:
#let's look at one

df = pd.read_csv('updated_processed_tracks_2021.csv')
df.head()

,track_id,track_name,artist_id,year,energy,danceability,instrumentalness,loudness,tempo,valence,energy_category,tempo_category,danceability_category,valence_category,danceability_valence_interaction,loudness_scaled,loudness_category,instrumentalness_category,custom_score
0,3cBsEDNhFI9E82vPj3kvi3,Wasted On You,4oUHIQIBe0LHzYfvXNW4QM,2021,0.657,0.505,0.00107,-5.240,196.000,0.252,Medium,Fast,Medium,Sad,0.127260,0.897591,Very High,Low,0.558268
1,69AIpwGNLxr4qS1X5ynx60,Buy Dirt,77kULmXAQ6vWer7IIHdGzI,2021,0.559,0.586,0.00000,-6.725,177.945,0.437,Medium,Fast,Medium,Neutral,0.256082,0.858925,Very High,None,0.596585
2,4S4QJfBGGrC8jRIjJHf1Ka,Pain,78rUTD7y6Cy67W1RVzYs7t,2021,0.617,0.829,0.00306,-8.497,125.605,0.810,Medium,Medium,High,Happy,0.671490,0.812785,Very High,Low,0.768007
3,3azJifCSqg9fRij2yKIbWz,The Color Violet,2jku7tDXc6XoB6MO2hFuqg,2021,0.534,0.645,0.00000,-10.800,105.020,0.463,Medium,Medium,Medium,Neutral,0.298635,0.752819,Very High,None,0.593314
4,5OELUCYgOHKFAvCERnAvfS,More Than My Hometown,4oUHIQIBe0LHzYfvXNW4QM,2021,0.869,0.620,0.00000,-5.479,126.019,0.597,High,Medium,Medium,Neutral,0.370140,0.891368,Very High,None,0.730774


The data looks good so next step is to check for duplicated track_id's 

In [9]:
# Let's check for duplicated track_id 

# Function to clean the tracks data for a given year

def clean_tracks_data(year):    
    # Define the file path
    tracks_file = f'updated_processed_tracks_{year}.csv'
    
    # Check if the file exists
    if os.path.exists(tracks_file):
        # Load the tracks data
        tracks_df = pd.read_csv(tracks_file)

        # Remove duplicates based on 'track_id'
        tracks_df = tracks_df.drop_duplicates(subset='track_id', keep='first')

        # Save the cleaned DataFrame to a new file
        output_file = f'updated_processedx_tracks_{year}.csv'
        tracks_df.to_csv(output_file, index=False)
        
        print(f"Saved cleaned file for {year} to {output_file}")
    else:
        print(f"File for {year} does not exist: {tracks_file}")

# Loop through the years 2015 to 2024
for year in range(2015, 2024 + 1):
    clean_tracks_data(year)


Saved cleaned file for 2015 to updated_processedx_tracks_2015.csv
Saved cleaned file for 2016 to updated_processedx_tracks_2016.csv
Saved cleaned file for 2017 to updated_processedx_tracks_2017.csv
Saved cleaned file for 2018 to updated_processedx_tracks_2018.csv
Saved cleaned file for 2019 to updated_processedx_tracks_2019.csv
Saved cleaned file for 2020 to updated_processedx_tracks_2020.csv
Saved cleaned file for 2021 to updated_processedx_tracks_2021.csv
Saved cleaned file for 2022 to updated_processedx_tracks_2022.csv
Saved cleaned file for 2023 to updated_processedx_tracks_2023.csv
Saved cleaned file for 2024 to updated_processedx_tracks_2024.csv


In [10]:
# Let's look at one

df1 = pd.read_csv('updated_processedx_tracks_2018.csv')
df1.head()

,track_id,track_name,artist_id,year,energy,danceability,instrumentalness,loudness,tempo,valence,energy_category,tempo_category,danceability_category,valence_category,danceability_valence_interaction,loudness_scaled,loudness_category,instrumentalness_category,custom_score
0,698eQRku24PIYPQPHItKlA,She Got the Best of Me,718COspgdWOnwOFpJHRZHS,2018,0.907,0.533,0.000000,-3.793,150.990,0.700,High,Fast,Medium,Happy,0.373100,0.934278,Very High,None,0.748506
1,2rxQMGVafnNaRaXlRMWPde,Beautiful Crazy,718COspgdWOnwOFpJHRZHS,2018,0.402,0.551,0.000000,-7.431,103.313,0.382,Medium,Medium,Medium,Neutral,0.210482,0.785308,Very High,None,0.518362
2,3GCdLUSnKSMJhs4Tj6CV3s,All The Stars (with SZA),2YZyLoL8N0Wb9xBt1NhZWg,2018,0.633,0.695,0.000195,-4.946,96.782,0.557,Medium,Slow,Medium,Neutral,0.387115,0.887064,Very High,Low,0.683413
3,7yNf9YjeO5JXUE3JEBgnYc,Babydoll,6USv9qhCn6zfxlBQIYJ9qs,2018,0.604,0.762,0.000000,-5.296,168.007,0.847,Medium,Fast,High,Happy,0.645414,0.872732,Very High,None,0.765896
4,57TEkZN3f84tFJFHFrD4pO,Whiskey Glasses,4oUHIQIBe0LHzYfvXNW4QM,2018,0.680,0.616,0.000001,-4.581,149.951,0.687,High,Fast,Medium,Happy,0.423192,0.902011,Very High,Low,0.706952


The data looks as expected.

In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   track_id                          400 non-null    object 
 1   track_name                        400 non-null    object 
 2   artist_id                         400 non-null    object 
 3   year                              400 non-null    int64  
 4   energy                            400 non-null    float64
 5   danceability                      400 non-null    float64
 6   instrumentalness                  400 non-null    float64
 7   loudness                          400 non-null    float64
 8   tempo                             400 non-null    float64
 9   valence                           400 non-null    float64
 10  energy_category                   400 non-null    object 
 11  tempo_category                    400 non-null    object 
 12  danceabi

In [12]:
# let's check the shapes of the tracks dataframes

# Function to read and print the shape of each file
def print_file_shapes(start_year, end_year):
    for year in range(start_year, end_year + 1):
        try:
            # Read the CSV file
            file_name = f'updated_processedx_tracks_{year}.csv'
            df = pd.read_csv(file_name)
            
            # Print the year and shape of the DataFrame
            print(f"Year: {year}, Shape: {df.shape}")
        
        except FileNotFoundError:
            print(f"File not found for year: {year}")

# Run the function for the years 2015 through 2024
print_file_shapes(2015, 2024)


Year: 2015, Shape: (398, 19)
Year: 2016, Shape: (399, 19)
Year: 2017, Shape: (397, 19)
Year: 2018, Shape: (400, 19)
Year: 2019, Shape: (400, 19)
Year: 2020, Shape: (399, 19)
Year: 2021, Shape: (392, 19)
Year: 2022, Shape: (394, 19)
Year: 2023, Shape: (400, 19)
Year: 2024, Shape: (400, 19)


It looks like a few dupicated tracks were removed

## Create SQLITE Database

The code below creates the database tables in sqlite and prints out the schema as a check

In [13]:
import sqlite3

# Define the database name
db_name = 'spotify_music.db'

conn = sqlite3.connect(f'{db_name}')
cursor = conn.cursor()

# Drop the existing 'artists' and 'tracks' tables if the exist - just a precaution
cursor.execute("DROP TABLE IF EXISTS artists;")
cursor.execute("DROP TABLE IF EXISTS tracks;")

# Define the schema for the 'artists' table 
create_artists_table = """
CREATE TABLE IF NOT EXISTS artists (
    artist_id TEXT PRIMARY KEY,
    artist_name TEXT,
    popularity INTEGER,
    genres TEXT,
    year INTEGER,
    simplified_genre TEXT
);
"""

# Define the schema for the 'tracks' table with the correct column names
create_tracks_table = """
CREATE TABLE IF NOT EXISTS tracks (
    track_id TEXT PRIMARY KEY,
    track_name TEXT,
    artist_id TEXT,
    year INTEGER,
    energy REAL,
    danceability REAL,
    instrumentalness REAL,
    loudness REAL,
    tempo REAL,
    valence REAL,
    energy_category TEXT,
    tempo_category TEXT,
    danceability_category TEXT,
    valence_category TEXT,
    danceability_valence_interaction REAL,
    loudness_scaled REAL,
    loudness_category TEXT,
    instrumentalness_category TEXT,
    custom_score REAL,
    FOREIGN KEY (artist_id) REFERENCES artists (artist_id)
);
"""

# Execute the table creation queries
cursor.execute(create_artists_table)
cursor.execute(create_tracks_table)

# Commit the changes
conn.commit()

# View the schema of the tables
print("Artists Table Schema:")
artists_schema = cursor.execute("PRAGMA table_info(artists);").fetchall()
for column in artists_schema:
    print(column)

print("\nTracks Table Schema:")
tracks_schema = cursor.execute("PRAGMA table_info(tracks);").fetchall()
for column in tracks_schema:
    print(column)

# Close the connection
conn.close()


Artists Table Schema:
(0, 'artist_id', 'TEXT', 0, None, 1)
(1, 'artist_name', 'TEXT', 0, None, 0)
(2, 'popularity', 'INTEGER', 0, None, 0)
(3, 'genres', 'TEXT', 0, None, 0)
(4, 'year', 'INTEGER', 0, None, 0)
(5, 'simplified_genre', 'TEXT', 0, None, 0)

Tracks Table Schema:
(0, 'track_id', 'TEXT', 0, None, 1)
(1, 'track_name', 'TEXT', 0, None, 0)
(2, 'artist_id', 'TEXT', 0, None, 0)
(3, 'year', 'INTEGER', 0, None, 0)
(4, 'energy', 'REAL', 0, None, 0)
(5, 'danceability', 'REAL', 0, None, 0)
(6, 'instrumentalness', 'REAL', 0, None, 0)
(7, 'loudness', 'REAL', 0, None, 0)
(8, 'tempo', 'REAL', 0, None, 0)
(9, 'valence', 'REAL', 0, None, 0)
(10, 'energy_category', 'TEXT', 0, None, 0)
(11, 'tempo_category', 'TEXT', 0, None, 0)
(12, 'danceability_category', 'TEXT', 0, None, 0)
(13, 'valence_category', 'TEXT', 0, None, 0)
(14, 'danceability_valence_interaction', 'REAL', 0, None, 0)
(15, 'loudness_scaled', 'REAL', 0, None, 0)
(16, 'loudness_category', 'TEXT', 0, None, 0)
(17, 'instrumentalness_ca

Looks good, now let's load the files to the database

In [14]:
# Database path
db_path = 'spotify_music.db'

# Function to load data for a given year
def load_data_for_year(year):
    # Generate file paths based on the year
    artist_file = f'processed_artist_{year}.csv'
    tracks_file = f'updated_processedx_tracks_{year}.csv'
    
    # Load the CSV files
    artists_df = pd.read_csv(artist_file)
    tracks_df = pd.read_csv(tracks_file)
    
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Insert data into the 'artists' table
    for _, row in artists_df.iterrows():
        cursor.execute("""
            INSERT OR IGNORE INTO artists (artist_id, artist_name, popularity, genres, year, simplified_genre)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (row['artist_id'], row['artist_name'], row['popularity'], row['genres'], row['year'], row['simplified_genre']))
    
    # Insert data into the 'tracks' table
    for _, row in tracks_df.iterrows():
        cursor.execute("""
            INSERT OR IGNORE INTO tracks (track_id, track_name, artist_id, year, energy, danceability, instrumentalness, loudness, tempo, valence,
                                          energy_category, tempo_category, danceability_category, valence_category, danceability_valence_interaction,
                                          loudness_scaled, loudness_category, instrumentalness_category, custom_score)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (row['track_id'], row['track_name'], row['artist_id'], row['year'], row['energy'], row['danceability'], row['instrumentalness'],
              row['loudness'], row['tempo'], row['valence'], row['energy_category'], row['tempo_category'], row['danceability_category'],
              row['valence_category'], row['danceability_valence_interaction'], row['loudness_scaled'], row['loudness_category'],
              row['instrumentalness_category'], row['custom_score']))
    
    # Commit the changes and close the connection
    conn.commit()
    conn.close()
    
    print(f"Data for {year} loaded successfully.")

# Loop through the years from 2015 to 2024 and load data
for year in range(2015, 2025):
    try:
        load_data_for_year(year)
    except FileNotFoundError:
        print(f"Data files for {year} not found. Skipping...")


Data for 2015 loaded successfully.
Data for 2016 loaded successfully.
Data for 2017 loaded successfully.
Data for 2018 loaded successfully.
Data for 2019 loaded successfully.
Data for 2020 loaded successfully.
Data for 2021 loaded successfully.
Data for 2022 loaded successfully.
Data for 2023 loaded successfully.
Data for 2024 loaded successfully.


Let's check the database dimensions

In [15]:

# Database path
db_path = 'spotify_music.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query to get the row count and column count of each table
artists_row_count = cursor.execute("SELECT COUNT(*) FROM artists;").fetchone()[0]
tracks_row_count = cursor.execute("SELECT COUNT(*) FROM tracks;").fetchone()[0]

artists_column_count = cursor.execute("PRAGMA table_info(artists);").fetchall()
tracks_column_count = cursor.execute("PRAGMA table_info(tracks);").fetchall()

# Close the connection
conn.close()

# Display the number of rows and columns
print(f"Artists Table: {artists_row_count} rows, {len(artists_column_count)} columns")
print(f"Tracks Table: {tracks_row_count} rows, {len(tracks_column_count)} columns")


Artists Table: 565 rows, 6 columns
Tracks Table: 3979 rows, 19 columns


### SQL Queries For Exploratory Data Analysis

### Let's Look at who the top 20 artists are along with their popularity score

In [16]:

# Function to execute a query and return the results
def execute_query(query):
    # Open the database connection
    conn = sqlite3.connect('spotify_music.db')
    cursor = conn.cursor()
    
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()
    
    # Close the database connection
    conn.close()
    
    return results

# Top Artists by Popularity
query = """
SELECT artist_name, popularity
FROM artists
ORDER BY popularity DESC
LIMIT 20;
"""

# Execute the query and print the results
results = execute_query(query)
print('The Top 20 Artists Are:')
for row in results:
    print(row)


The Top 20 Artists Are:
('Taylor Swift', 100)
('Drake', 95)
('Billie Eilish', 95)
('The Weeknd', 94)
('Bad Bunny', 94)
('Eminem', 93)
('Travis Scott', 93)
('Kendrick Lamar', 92)
('Kanye West', 92)
('Peso Pluma', 92)
('Morgan Wallen', 91)
('Future', 91)
('Post Malone', 91)
('Lana Del Rey', 91)
('Ariana Grande', 91)
('KAROL G', 91)
('Feid', 91)
('Zach Bryan', 91)
('Sabrina Carpenter', 90)
('Rihanna', 90)


### Let's look at the top genres based on artist simplified genre.

In [17]:
# Function to execute a query and return the results
def execute_query(query):
    # Open the database connection
    conn = sqlite3.connect('spotify_music.db')
    cursor = conn.cursor()
    
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()
    
    # Close the database connection
    conn.close()
    
    return results

# Simplified Genre Distribution based on artist simplified genre
query = """
SELECT simplified_genre, COUNT(*) AS genre_count
FROM artists
GROUP BY simplified_genre
ORDER BY genre_count DESC;

"""

# Execute the query and print the results
results = execute_query(query)
print('The Top Simplified Genres Are:')
for row in results:
    print(row)


The Top Simplified Genres Are:
('Pop', 156)
('Hip Hop', 84)
('Other', 68)
('Rock', 58)
('Rap', 51)
('Country', 42)
('Metal', 29)
('Indie', 18)
('R&B', 17)
('Reggae', 16)
('Soul', 10)
('House', 7)
('Funk', 5)
('Noise', 1)
('Lo-Fi', 1)
('Grunge', 1)
('Anime', 1)


### Let's look at some average track features based on simplified genre

In [18]:
import sqlite3
import pandas as pd

# Database path
db_path = 'spotify_music.db'

# SQL query
query = """
SELECT simplified_genre,
       AVG(energy) AS avg_energy,
       AVG(danceability) AS avg_danceability,
       AVG(tempo) AS avg_tempo,
       AVG(valence) AS avg_valence,
       AVG(loudness_scaled) AS avg_loudness_scaled
FROM tracks
JOIN artists ON tracks.artist_id = artists.artist_id
GROUP BY simplified_genre
ORDER BY simplified_genre;
"""

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Execute the query and load the result into a DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame with good formatting
print(df.to_string(index=False))  # Using `to_string` for plain text output with no index column


simplified_genre  avg_energy  avg_danceability  avg_tempo  avg_valence  avg_loudness_scaled
           Anime    0.869000          0.599000 132.889000     0.453000             0.899310
         Country    0.629135          0.568509 122.065532     0.490981             0.809749
         Hip Hop    0.597005          0.719095 122.804647     0.422027             0.795977
           House    0.767429          0.680571 129.798143     0.487657             0.880863
           Indie    0.532518          0.580307 118.177342     0.478041             0.760292
           Metal    0.836205          0.474667 132.662231     0.368297             0.880670
           Other    0.592365          0.616022 125.525762     0.542178             0.750215
             Pop    0.595957          0.635806 120.661711     0.462853             0.793486
             R&B    0.513081          0.614883 117.295162     0.382696             0.760354
             Rap    0.643151          0.666652 124.985379     0.424771          

General Observations:
Anime and Metal are characterized by high energy, high tempo, and high loudness, but differ significantly in valence, with Anime being more positive and Metal being more negative.

Reggae and House genres are highly danceable, but Reggae has a lower tempo and higher valence, making it more relaxed and upbeat.

Pop and Hip Hop share similar average values across all features, reflecting their broad appeal in today's music.

R&B consistently shows lower energy, tempo, and valence, indicative of it's emotional depth and slower, more soulful rhythms.

#### Query for What tracks by are most popular and could be used for marketing ?

In [19]:
# Function to execute a query and return the results
def execute_query(query):
    # Open the database connection
    conn = sqlite3.connect('spotify_music.db')
    cursor = conn.cursor()
    
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()
    
    # Close the database connection
    conn.close()
    
    return results

# Simplified Genre Distribution based on artist simplified genre
query = """
SELECT track_name, artist_name, popularity
FROM tracks
JOIN artists ON tracks.artist_id = artists.artist_id
ORDER BY popularity DESC
LIMIT 20;

"""

# Execute the query and print the results
results = execute_query(query)
print('The Top Tracks Are:')
for row in results:
    print(row)


The Top Tracks Are:
('Don’t Blame Me', 'Taylor Swift', 100)
('...Ready For It?', 'Taylor Swift', 100)
('Delicate', 'Taylor Swift', 100)
('Look What You Made Me Do', 'Taylor Swift', 100)
('Getaway Car', 'Taylor Swift', 100)
('I Did Something Bad', 'Taylor Swift', 100)
('Gorgeous', 'Taylor Swift', 100)
('Dress', 'Taylor Swift', 100)
('End Game', 'Taylor Swift', 100)
('Call It What You Want', 'Taylor Swift', 100)
('King Of My Heart', 'Taylor Swift', 100)
('Dancing With Our Hands Tied', 'Taylor Swift', 100)
("This Is Why We Can't Have Nice Things", 'Taylor Swift', 100)
('New Year’s Day', 'Taylor Swift', 100)
('So It Goes...', 'Taylor Swift', 100)
('Cruel Summer', 'Taylor Swift', 100)
('Lover', 'Taylor Swift', 100)
('The Man', 'Taylor Swift', 100)
('You Need To Calm Down', 'Taylor Swift', 100)
('Miss Americana & The Heartbreak Prince', 'Taylor Swift', 100)


OK, so this query allows Taylor Swift to dominate, so I will modify it to select more artists and their songs

In [60]:
import sqlite3
import pandas as pd

# Database path
db_path = 'spotify_music.db'

# SQL query
query = """
WITH RankedTracks AS (
    SELECT 
        track_name, 
        artist_name, 
        popularity,
        simplified_genre,
        ROW_NUMBER() OVER (PARTITION BY artist_name ORDER BY popularity DESC) AS rank
    FROM tracks
    JOIN artists ON tracks.artist_id = artists.artist_id
)
SELECT track_name, artist_name, popularity, simplified_genre
FROM RankedTracks
WHERE rank = 1
ORDER BY popularity DESC
LIMIT 50;
"""

# Open the database connection
conn = sqlite3.connect(db_path)

# Adjust display options for better readability on the same line
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# Execute the query and load the result into a DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


                       track_name                        artist_name      popularity simplified_genre
0                                      Don’t Blame Me       Taylor Swift      100             Pop    
1                                          ocean eyes      Billie Eilish       95             Pop    
2                                              Jungle              Drake       95         Hip Hop    
3                                               Diles          Bad Bunny       94          Reggae    
4                                           The Hills         The Weeknd       94             R&B    
5                            River (feat. Ed Sheeran)             Eminem       93         Hip Hop    
6                             90210 (feat. Kacy Hill)       Travis Scott       93             Rap    
7                       Father Stretch My Hands Pt. 1         Kanye West       92             Rap    
8                                             Alright     Kendrick Lamar       92 

####  Let's do a query to determine which track feautres are most correlated with popularity if any

In [21]:
import pandas as pd
import sqlite3

# Database path
db_path = 'spotify_music.db'  

# SQL query to get the data
query = """
SELECT energy, danceability, valence, tempo, popularity
FROM tracks
JOIN artists ON tracks.artist_id = artists.artist_id;
"""

# Function to execute a query and return the results as a DataFrame

def execute_query_to_df(query):
    
    # Open the database connection
    conn = sqlite3.connect(db_path)
    
    # Execute the query and load the result into a DataFrame
    df = pd.read_sql_query(query, conn)
    
    # Close the database connection
    conn.close()
    
    return df

# Execute the query and load the data into a DataFrame
df = execute_query_to_df(query)

# Calculate the correlation matrix
correlation_matrix = df.corr()

# Display the correlation matrix
print(correlation_matrix)


                energy  danceability   valence     tempo  popularity
energy        1.000000      0.083051  0.418968  0.114315   -0.052302
danceability  0.083051      1.000000  0.312063 -0.033040   -0.017161
valence       0.418968      0.312063  1.000000  0.041445   -0.081890
tempo         0.114315     -0.033040  0.041445  1.000000   -0.017630
popularity   -0.052302     -0.017161 -0.081890 -0.017630    1.000000


 - The correlations between popularity and the other features (energy, danceability, valence, tempo) are all very weak (close to zero), indicating that none of these features have a strong influence on the popularity of a track in this dataset.

- Valence has the strongest relationship with Energy (0.419), meaning that tracks with higher energy levels are somewhat more likely to be positive or happy.

- Popularity does not seem to have strong correlations with any of the features, suggesting that other factors (not captured by these features) might be driving a track's popularity.

Conclusion:
Most of the relationships between these features are weak, indicating that each feature is relatively independent of the others.

#### Let's create a possible playlist based on user preferences

In [22]:

# Function to execute a query and return the results
def execute_query(query):
    # Open the database connection
    conn = sqlite3.connect('spotify_music.db')
    cursor = conn.cursor()
    
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()
    
    # Close the database connection
    conn.close()
    
    return results

# Query to select tracks based on a user's preferences
query = """
SELECT track_name, artist_name, energy, danceability, valence
FROM tracks
JOIN artists ON tracks.artist_id = artists.artist_id
WHERE energy BETWEEN 0.7 AND 0.9
AND valence > 0.5
ORDER BY danceability DESC
LIMIT 30;
"""

# Execute the query and get the results
results = execute_query(query)

# Print headers
print(f"{'Track Name':<50} {'Artist Name':<25} {'Energy':<10} {'Danceability':<15} {'Valence':<10}")

# Print the results with formatting
for row in results:
    track_name, artist_name, energy, danceability, valence = row
    print(f"{track_name:<50} {artist_name:<25} {energy:<10.2f} {danceability:<15.2f} {valence:<10.2f}")


Track Name                                         Artist Name               Energy     Danceability    Valence   
Climate (feat. Offset)                             BigXthaPlug               0.75       0.96            0.70      
Houdini                                            Eminem                    0.89       0.94            0.89      
Houdini                                            Eminem                    0.88       0.94            0.90      
Big Energy                                         Latto                     0.81       0.94            0.81      
Broke In A Minute                                  Tory Lanez                0.70       0.93            0.80      
Chk Chk Boom                                       Stray Kids                0.74       0.93            0.84      
Go Stupid                                          Polo G                    0.83       0.92            0.65      
4K                                                 El Alfa                   0.8

### Let's Look at a query to determine top artists in emerging genres

In [23]:

# Function to execute a query and return the results
def execute_query(query):
    # Open the database connection
    conn = sqlite3.connect('spotify_music.db')
    cursor = conn.cursor()
    
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()
    
    # Close the database connection
    conn.close()
    
    return results

query = """SELECT artist_name, simplified_genre, popularity
FROM artists
WHERE simplified_genre IN ('Indie', 'Other', 'House')  
ORDER BY popularity DESC
LIMIT 20;
"""

# Execute the query and print the results
results = execute_query(query)
print('Artist', '    ' , 'Genre', '    ', 'Popularity')
for row in results:
    print(row)



Artist      Genre      Popularity
('Peso Pluma', 'Other', 92)
('Arijit Singh', 'Other', 90)
('Junior H', 'Other', 89)
('Natanael Cano', 'Other', 88)
('Pritam', 'Other', 87)
('Fuerza Regida', 'Other', 87)
('Carin Leon', 'Other', 86)
('Noah Kahan', 'Indie', 85)
('Disney', 'Other', 84)
('Tiësto', 'House', 84)
('Luis R Conriquez', 'Other', 84)
('Oscar Maydon', 'Other', 84)
('Grupo Frontera', 'Other', 84)
('Mitski', 'Indie', 83)
('Gabito Ballesteros', 'Other', 83)
('Central Cee', 'Other', 83)
('Gracie Abrams', 'Other', 83)
('Teddy Swims', 'Other', 83)
('¥$', 'Other', 83)
('TV Girl', 'Indie', 82)


This list above shows artists who have high popularity scores but are not in mainstream genres

### Let's look at Under-Rated Tracks With High Custom Scores 

In [24]:

# Function to execute a query and return the results
def execute_query(query):
    # Open the database connection
    conn = sqlite3.connect('spotify_music.db')
    cursor = conn.cursor()
    
    # Execute the query
    cursor.execute(query)
    results = cursor.fetchall()
    
    # Close the database connection
    conn.close()
    
    return results

query = """SELECT track_name, artist_name, custom_score, popularity
FROM tracks
JOIN artists ON tracks.artist_id = artists.artist_id
WHERE artists.popularity < 70  
ORDER BY custom_score DESC
LIMIT 20;

"""

# Execute the query and get the results
results = execute_query(query)

# Check if results were returned and print them
if results:
    print(f"{'Track Name':<30} {'Artist Name':<25} {'Custom Score':<15} {'Popularity':<10}")
    for row in results:
        track_name, artist_name, custom_score, popularity = row
        print(f"{track_name:<30} {artist_name:<25} {custom_score:<15.2f} {popularity:<10}")
else:
    print("No results found. Please check if your query conditions match any records.")


Track Name                     Artist Name               Custom Score    Popularity
Erbody But Me                  Tech N9ne                 0.81            69        
Beers On Me                    Dierks Bentley            0.76            68        
Handle On You                  Parker McCollum           0.75            67        
23                             Sam Hunt                  0.74            69        
From The D To The A (feat. Lil Yachty) Tee Grizzley              0.74            69        
Lavender Girl                  Caamp                     0.74            69        
Burning Man                    Dierks Bentley            0.67            68        
Body Like A Back Road          Sam Hunt                  0.66            69        
Loop Hole (feat. 21 Savage)    Tee Grizzley              0.65            69        
Body Like A Back Road          Sam Hunt                  0.64            69        
Pretty Heart                   Parker McCollum           0.62       

Sam Hunt, Dierks Bentley, Parker McCollum, Tee Grizzley, and Caamp are the most frequently occurring artists in the list. This suggests that these artists have multiple tracks with a custom_score that ranks high, even though their popularity scores are relatively modest (in the high 60s).

Sam Hunt, Dierks Bentley, Parker McCollum,are country artists who may not necessarily reach mainstream popularity, but have dedicated fan bases

Artists with moderate popularity but high custom scores might be ideal candidates for niche marketing campaigns. These tracks might perform well in curated playlists targeted at specific listener groups.

Tracks with high custom scores but lower popularity may benefit from additional promotion to increase their reach, potentially moving them up the popularity scale.



### Which tracks have the highest danceability ?

In [57]:

# Create a connection to the SQLite database
conn = sqlite3.connect('spotify_music.db')
cursor = conn.cursor()

# Define a query to find the top 20 tracks with the highest danceability
# Join the TRACKS and ARTISTS tables on the artist_id
query = """
SELECT t.track_name, t.danceability, t.danceability_category, a.artist_name, a.simplified_genre
FROM TRACKS t
JOIN ARTISTS a ON t.artist_id = a.artist_id
ORDER BY t.danceability DESC
LIMIT 20;
"""

# Execute the query
cursor.execute(query)

# Fetch the result
top_20tracks = cursor.fetchall()


# Convert the result to a DataFrame for better readability
df = pd.DataFrame(top_20tracks, columns=["track_name", "danceability", "danceability_category", "artist_name", "simplified_genre"])

# Adjust display options for better readability on the same line
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# Print the DataFrame
print(df)


# Close the database connection
conn.close()




                 track_name               danceability danceability_category       artist_name       simplified_genre
0           Bad Bad Bad (feat. Lil Baby)      0.974             High                      Young Thug      Hip Hop    
1   Johnny Dang (with Paul Wall & DRODi)      0.971             High                 That Mexican OT          Rap    
2                              Safehouse      0.967             High                     BigXthaPlug      Hip Hop    
3                           Pure Cocaine      0.964             High                        Lil Baby      Hip Hop    
4                             Yes Indeed      0.963             High                        Lil Baby      Hip Hop    
5                               Low Down      0.962             High                        Lil Baby      Hip Hop    
6                 Climate (feat. Offset)      0.960             High                     BigXthaPlug      Hip Hop    
7                             ATTENTION!      0.954     

### Which tracks have the lowest danceability ?

In [56]:
# Create a connection to the SQLite database
conn = sqlite3.connect('spotify_music.db')
cursor = conn.cursor()

# Define a query to find the top 20 tracks with the highest danceability
# Join the TRACKS and ARTISTS tables on the artist_id
query = """
SELECT t.track_name, t.danceability, t.danceability_category, a.artist_name, a.simplified_genre
FROM TRACKS t
JOIN ARTISTS a ON t.artist_id = a.artist_id
ORDER BY t.danceability ASC
LIMIT 20;
"""

# Execute the query
cursor.execute(query)

# Fetch the result
top_20tracks = cursor.fetchall()


# Convert the result to a DataFrame for better readability
df = pd.DataFrame(top_20tracks, columns=["track_name", "danceability", "danceability_category", "artist_name", "simplified_genre"])

# Adjust display options for better readability on the same line
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# Print the DataFrame
print(df)


# Close the database connection
conn.close()



                       track_name                      danceability danceability_category      artist_name      simplified_genre
0   Rain Sounds for Sleeping, Pt. 07 (Continuous N...     0.0602              Low          White Noise Radiance       Other     
1   Rain Sounds for Sleeping, Pt. 04 (Continuous N...     0.0608              Low          White Noise Radiance       Other     
2   Rain Sounds for Sleeping, Pt. 02 (Continuous N...     0.1070              Low          White Noise Radiance       Other     
3   Rain Sounds for Sleeping, Pt. 01 (Continuous N...     0.1080              Low          White Noise Radiance       Other     
4   Rain Sounds for Sleeping, Pt. 03 (Continuous N...     0.1150              Low          White Noise Radiance       Other     
5                528 Hz Bring Positive Transformation     0.1230              Low                 Miracle Tones       Other     
6   Rain Sounds for Sleeping, Pt. 08 (Continuous N...     0.1430              Low          White 

### Which tracks have the highest tempo ?

In [55]:
# Create a connection to the SQLite database

conn = sqlite3.connect('spotify_music.db')
cursor = conn.cursor()

# Define a query to find the top 20 tracks with the highest tempo


query = """
SELECT t.track_name, t.tempo, t.tempo_category, a.artist_name, a.simplified_genre
FROM TRACKS t
JOIN ARTISTS a ON t.artist_id = a.artist_id
ORDER BY t.tempo DESC
LIMIT 20;
"""


# Execute the query
cursor.execute(query)

# Fetch the result
top_20tracks = cursor.fetchall()

# Convert the result to a DataFrame for better readability
df = pd.DataFrame(top_20tracks, columns=["track_name", "tempo", "tempo_category", "artist_name", "simplified_genre"])

# Adjust display options for better readability on the same line
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# Print the DataFrame
print(df)


# Close the database connection
conn.close()

                track_name               tempo  tempo_category      artist_name      simplified_genre
0                             Mama Cry  220.099      Fast                  YNW Melly          Rap    
1   Lord Pretty Flacko Jodye 2 (LPFJ2)  207.982      Fast                 A$AP Rocky      Hip Hop    
2                           Bubble Gum  207.970      Fast                     Clairo          Pop    
3                      FourFiveSeconds  205.846      Fast                    Rihanna          Pop    
4                              Love Me  204.132      Fast                  Lil Tecca          Rap    
5                            7 Summers  203.903      Fast              Morgan Wallen      Country    
6                           Last Night  203.812      Fast              Morgan Wallen      Country    
7                           Last Night  203.812      Fast              Morgan Wallen      Country    
8                           Before You  202.899      Fast               Benson Boo

### Which tracks have the slowest tempo ?

In [54]:
# Create a connection to the SQLite database

conn = sqlite3.connect('spotify_music.db')
cursor = conn.cursor()

# Define a query to find the top 20 tracks with the highest tempo


query = """
SELECT t.track_name, t.tempo, t.tempo_category, a.artist_name, a.simplified_genre
FROM TRACKS t
JOIN ARTISTS a ON t.artist_id = a.artist_id
ORDER BY t.tempo ASC
LIMIT 20;
"""


# Execute the query
cursor.execute(query)

# Fetch the result
top_20tracks = cursor.fetchall()

# Convert the result to a DataFrame for better readability
df = pd.DataFrame(top_20tracks, columns=["track_name", "tempo", "tempo_category", "artist_name","simplified_genre"])

# Adjust display options for better readability on the same line
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# Print the DataFrame
print(df)


# Close the database connection
conn.close()

                       track_name                       tempo tempo_category      artist_name      simplified_genre
0                                   Tennessee Whiskey  48.718      Slow            Chris Stapleton      Country    
1                     Ocean Prime (feat. Boldy James)  49.305      Slow              The Alchemist      Hip Hop    
2                                                  J.  51.660      Slow               Ivan Cornejo        Other    
3                                           FRI(END)S  53.376      Slow                          V        Other    
4                                      hotline (edit)  56.829      Slow              Billie Eilish          Pop    
5                       Congratulations (feat. Bilal)  57.750      Slow                 Mac Miller      Hip Hop    
6                         Destroy Myself Just For You  59.677      Slow               Montell Fish          Rap    
7                                             Element  61.311      Slow 

### Which tracks are the loudest ?

In [53]:
# Create a connection to the SQLite database

conn = sqlite3.connect('spotify_music.db')
cursor = conn.cursor()

# Define a query to find the top 20 tracks with the highest tempo

query = """
SELECT t.track_name, t.loudness_scaled, t.loudness_category, a.artist_name, a.simplified_genre
FROM TRACKS t
JOIN ARTISTS a ON t.artist_id = a.artist_id
ORDER BY t.loudness_scaled DESC
LIMIT 20;
"""


# Execute the query
cursor.execute(query)

# Fetch the result
top_20tracks = cursor.fetchall()

# Convert the result to a DataFrame for better readability
df = pd.DataFrame(top_20tracks, columns=["track_name", "loudness_scaled", "loudness_category", "artist_name","simplified_genre"])

# Adjust display options for better readability on the same line
pd.set_option('display.colheader_justify', 'center')  # Center the column headers
pd.set_option('display.width', 1000)  # Adjust the width of the display
pd.set_option('display.max_columns', None)

# Print the DataFrame
print(df.to_string(index=False))


# Close the database connection
conn.close()

                       track_name                        loudness_scaled loudness_category     artist_name     simplified_genre
                                           Church Bells     1.000000         Very High        Carrie Underwood     Country     
                                 through the late night     1.000000         Very High            Travis Scott         Rap     
                                                 Bonita     1.000000         Very High                J Balvin      Reggae     
                                                 Uproar     1.000000         Very High               Lil Wayne     Hip Hop     
                                               Brickell     1.000000         Very High                    Feid         Pop     
                                   Memoirs of a Gorilla     0.998850         Very High             $uicideboy$         Rap     
                                                FRIENDS     0.991974         Very High              Mars

### Which tracks are the softest or lowest ?

In [51]:
# Create a connection to the SQLite database

conn = sqlite3.connect('spotify_music.db')
cursor = conn.cursor()

# Define a query to find the top 20 tracks with the highest tempo

query = """
SELECT t.track_name, t.loudness_scaled, t.loudness_category, a.artist_name, a.simplified_genre
FROM TRACKS t
JOIN ARTISTS a ON t.artist_id = a.artist_id
ORDER BY t.loudness_scaled ASC
LIMIT 20;
"""


# Execute the query
cursor.execute(query)

# Fetch the result
top_20tracks = cursor.fetchall()

# Convert the result to a DataFrame for better readability
df = pd.DataFrame(top_20tracks, columns=["track_name", "loudness_scaled", "loudness_category", "artist_name","simplified_genre"])

# Adjust display options for better readability on the same line
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# Print the DataFrame
print(df)


# Close the database connection
conn.close()

                                           track_name  loudness_scaled loudness_category      artist_name simplified_genre
0                                528 Hz Manifest Love         0.000000          Very Low    Miracle Tones            Other
1                                           Bass Boat         0.000000          Very Low       Zach Bryan          Country
2                      528 Hz Manifest Your Greatness         0.001904          Very Low    Miracle Tones            Other
3                                              SKINNY         0.037174          Very Low    Billie Eilish              Pop
4                                         BITTERSUITE         0.042294          Very Low    Billie Eilish              Pop
5                528 Hz Bring Positive Transformation         0.067070          Very Low    Miracle Tones            Other
6                              528 Hz Love & Miracles         0.075637          Very Low    Miracle Tones            Other
7               

### Which tracks have the highest valence ( mood ) ?

In [50]:
# Create a connection to the SQLite database

conn = sqlite3.connect('spotify_music.db')
cursor = conn.cursor()

# Define a query to find the top 20 tracks with the highest tempo

query = """
SELECT t.track_name, t.valence, t.valence_category, a.artist_name, a.simplified_genre
FROM TRACKS t
JOIN ARTISTS a ON t.artist_id = a.artist_id
ORDER BY t.valence DESC
LIMIT 20;
"""


# Execute the query
cursor.execute(query)

# Fetch the result
top_20tracks = cursor.fetchall()

# Convert the result to a DataFrame for better readability
df = pd.DataFrame(top_20tracks, columns=["track_name", "valence", "valence_category", "artist_name", "simplified_genre"])

# Adjust display options for better readability on the same line
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# Print the DataFrame
print(df)


# Close the database connection
conn.close()

                             track_name  valence valence_category                        artist_name simplified_genre
0                                   Ryd    0.980            Happy                         Steve Lacy            Other
1                 Pongámonos De Acuerdo    0.976            Happy  Julión Álvarez y su Norteño Banda            Other
2                           El Paciente    0.973            Happy                     Alfredo Olivas            Other
3                                  Doja    0.972            Happy                        Central Cee            Other
4       There's Nothing Holdin' Me Back    0.969            Happy                       Shawn Mendes              Pop
5                                  Uuuu    0.968            Happy                         Steve Lacy            Other
6                         El Hijo Mayor    0.967            Happy                           Junior H            Other
7                       Body Dysmorphia    0.966        

### Which tracks have the lowest valence ( mood ) ?

In [49]:
# Create a connection to the SQLite database

conn = sqlite3.connect('spotify_music.db')
cursor = conn.cursor()

# Define a query to find the top 20 tracks with the highest tempo

query = """
SELECT t.track_name, t.valence, t.valence_category, a.artist_name, a.simplified_genre
FROM TRACKS t
JOIN ARTISTS a ON t.artist_id = a.artist_id
ORDER BY t.valence ASC
LIMIT 20;
"""


# Execute the query
cursor.execute(query)

# Fetch the result
top_20tracks = cursor.fetchall()

# Convert the result to a DataFrame for better readability
df = pd.DataFrame(top_20tracks, columns=["track_name", "valence", "valence_category", "artist_name", "simplified_genre"])

# Adjust display options for better readability on the same line
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# Print the DataFrame
print(df)


# Close the database connection
conn.close()

                                           track_name  valence valence_category           artist_name simplified_genre
0   Rain Sounds for Sleeping, Pt. 01 (Continuous N...  0.00001              Sad  White Noise Radiance            Other
1   Rain Sounds for Sleeping, Pt. 02 (Continuous N...  0.00001              Sad  White Noise Radiance            Other
2   Rain Sounds for Sleeping, Pt. 03 (Continuous N...  0.00001              Sad  White Noise Radiance            Other
3   Rain Sounds for Sleeping, Pt. 04 (Continuous N...  0.00001              Sad  White Noise Radiance            Other
4   Rain Sounds for Sleeping, Pt. 05 (Continuous N...  0.00001              Sad  White Noise Radiance            Other
5   Rain Sounds for Sleeping, Pt. 06 (Continuous N...  0.00001              Sad  White Noise Radiance            Other
6   Rain Sounds for Sleeping, Pt. 07 (Continuous N...  0.00001              Sad  White Noise Radiance            Other
7   Rain Sounds for Sleeping, Pt. 08 (Continuous

### END of Note Book

### Summary

This notebook completed the following tasks:
 - performed final data cleaning
 - created SQLite database "spotify_music"
 - loaded artist and track data into the database
 - performed basic EDA on the data after loading using SQL queries